# ALPACA-inspired ROI transformation
performs an inverse transformation (from MNI reference to participant's native space) of auditory cortex
ROIs extracted from publicly available atlases

In [1]:
##### Import important modules #####

from os.path import join as opj
from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces.freesurfer import FSCommand, Binarize, Label2Label
from nipype.interfaces.utility import Function, IdentityInterface, Merge
from nipype.interfaces.io import SelectFiles, FreeSurferSource, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode

/home/mirjam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/derivatives/mindboggle/freesurfer_subjects'
FSCommand.set_default_subjects_dir(fs_dir)

# Specify variables

In [23]:
##### Specify important variables #####

experiment_dir = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS' # location of experiment folder
input_dir_preproc = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/preprocessing/output_preproc' # name of preprocessing output folder
input_dir_reg = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/preprocessing/output_reg' # name of registration output folder
input_dir_ROI = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/harvard-oxford_ROIs' # location of ROI folder

subject_list = ['sub-00'] # create the subject_list variable

output_dir = 'plum/output_inverse_transform_ROIs'  # name of norm output folder
working_dir = 'plum/workingdir_inverse_transform_ROIs'  # name of norm working directory

# Specify Nodes

In [24]:
# Transform the volumetric ROIs to the target space
inverse_transform_rois = MapNode(ApplyTransforms(args='--float',
                                  input_image_type=3,
                                  interpolation='Linear',
                                  invert_transform_flags=[False],
                                  num_threads=1,
                                  terminal_output='file'),
                  name='inverse_transform_rois', iterfield=['input_image'])

# Specify input and output stream

In [25]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
inverse_transform_composite = opj(input_dir_reg, 'registrationtemp', '_subject_id_{subject_id}', 'transformInverseComposite.h5')
target = opj(input_dir_preproc, 'mriconvert', '_subject_id_{subject_id}', 'brain_out.nii.gz')
ROIs = opj(input_dir_ROI, '*.nii.gz')
#havox_HG_l = opj(input_dir_ROI, '/harvard_oxford_HG_l.nii.gz')
#havox_HG_r = opj(input_dir_ROI, '/harvard_oxford_HG_r.nii.gz')
#havox_STGa_l = opj(input_dir_ROI, '/harvard_oxford_STGa_l.nii.gz')
#havox_STGa_r = opj(input_dir_ROI, '/harvard_oxford_STGa_r.nii.gz')
#havox_STGp_l = opj(input_dir_ROI, '/harvard_oxford_STGp_l.nii.gz')
#havox_STGp_r = opj(input_dir_ROI, '/harvard_oxford_STGp_r.nii.gz')

templates = {'inverse_transform_composite' : inverse_transform_composite,
             'target': target,
             'ROIs' : ROIs
             #'havox_HG_l' : havox_HG_l,
             #'havox_HG_r' : havox_HG_r,
             #'havox_STGa_l' : havox_STGa_l,
             #'havox_STGa_r' : havox_STGa_r,
             #'havox_STGp_l' : havox_STGp_l,
             #'havox_STGp_r' : havox_STGp_r
            }

selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                 ('_apply2con', 'apply2con'),
                 ('_warpall', 'warpall'),
                 ('_mask_', '')]
datasink.inputs.substitutions = substitutions

# initiate workflow and connect nodes

In [26]:
# Initiation of the inverse transform ROIs workflow
inverse_ROI_flow = Workflow(name='inverse_ROI_flow')
inverse_ROI_flow.base_dir = opj(experiment_dir, working_dir)

# Connect SelectFiles and DataSink to the workflow
inverse_ROI_flow.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (selectfiles, inverse_transform_rois, [('ROIs', 'input_image')]),
                  (selectfiles, inverse_transform_rois, [('target', 'reference_image')]),
                  (selectfiles, inverse_transform_rois, [('inverse_transform_composite', 'transforms')]),
                          
                  (inverse_transform_rois, datasink, [('output_image', 'inverse_transform_rois.@roitransform')]),
                         ])

# visualize the pipeline

In [27]:
# Create a colored output graph
inverse_ROI_flow.write_graph(graph2use='colored',format='png', simple_form=True)

# Create a detailed output graph
inverse_ROI_flow.write_graph(graph2use='flat',format='png', simple_form=True)

180806-11:36:42,372 workflow INFO:
	 Generated workflow graph: /home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/plum/workingdir_inverse_transform_ROIs/inverse_ROI_flow/graph.png (graph2use=colored, simple_form=True).
180806-11:36:42,571 workflow INFO:
	 Generated workflow graph: /home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/plum/workingdir_inverse_transform_ROIs/inverse_ROI_flow/graph.png (graph2use=flat, simple_form=True).


'/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/plum/workingdir_inverse_transform_ROIs/inverse_ROI_flow/graph.png'

# run workflow

In [28]:
#### run the workflow using multiple cores ####
inverse_ROI_flow.run('MultiProc', plugin_args={'n_procs':4})

180806-11:36:45,732 workflow INFO:
	 Workflow inverse_ROI_flow settings: ['check', 'execution', 'logging', 'monitoring']
180806-11:36:45,799 workflow INFO:
	 Running in parallel.
180806-11:36:45,814 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 10.45/10.45, Free processors: 4/4.
180806-11:36:45,880 workflow INFO:
	 [Node] Setting-up "inverse_ROI_flow.selectfiles" in "/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/plum/workingdir_inverse_transform_ROIs/inverse_ROI_flow/_subject_id_sub-00/selectfiles".
180806-11:36:45,888 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
180806-11:36:45,937 workflow WARNING:
	 [Node] Error on "inverse_ROI_flow.selectfiles" (/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/plum/workingdir_inverse_transform_ROIs/inverse_ROI_flow/_subject_id_sub-00/selectfiles)
180806-11:36:47,817 workflow ERROR:
	 Node selectfiles.a0 failed to run on host mirjam-X556UQK.
180806-11:36:47,821 workflow ERROR:
	 Savi

RuntimeError: Workflow did not execute cleanly. Check log for details